In [10]:
#Import libraries
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import PIL as pil
import os
import PIL
import pathlib
import glob
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from sklearn.utils import shuffle


#Set to GPU
devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(devices[0], True)

#Path
images_source = pathlib.Path("D:\s4532664\COMP3710\ISIC2018_Task1-2_Training_Data")

#Input images
input_dir = images_source / "ISIC2018_Task1-2_Training_Input_x2\*.jpg"

#Segmentation images (label images)
seg_dir = images_source / "ISIC2018_Task1_Training_GroundTruth_x2\*.png"
                             
#Setup arrays for training, testing and validation datasets
X_train = None
X_test = None
X_valid = None
y_train = None
y_test = None
y_valid = None

training = None
validation = None
testing = None

In [11]:
print(input_dir)
print(seg_dir)

D:\s4532664\COMP3710\ISIC2018_Task1-2_Training_Data\ISIC2018_Task1-2_Training_Input_x2\*.jpg
D:\s4532664\COMP3710\ISIC2018_Task1-2_Training_Data\ISIC2018_Task1_Training_GroundTruth_x2\*.png


In [21]:
#Process every filename in the array as an image, using TF functions. 
#Normalise every image by either 0 or 1 and divide it by 255 for scaling purposes

def convert_file_image(input_file, seg_file):
   
    input_image = tf.io.read_file(input_file)
    input_image = tf.image.decode_jpeg(input_image, channels = 1)
    input_image = tf.image.resize(input_image, [256, 256])
    input_image = tf.cast(input_image, tf.float32) / 255.0
    
    seg_image = tf.io.read_file(seg_file)
    seg_image = tf.image.decode_png(seg_image, channels = 1)
    seg_image = tf.image.resize(seg_image, [256, 256])
    seg_image = seg_image == [0, 255]
    seg_image = tf.cast(seg_image, tf.float32)
    
    return input_image, seg_image

In [22]:
#Load images from disk into training, testing and validation arrays (DO DOCSTRING)
def load_data(inputdir, segdir):
    
    xtrain = sorted(glob.glob(str(inputdir)))
    ytrain = sorted(glob.glob(str(segdir)))
    
    xtrain, ytrain = shuffle(xtrain, labels)
    
    #Split the images into 3 datasets (Training - 50%, Validation - 25%, Testing - 25%)
    
    half_length = int(len(xtrain)/2)
    quarter_length_ceil = int(tf.math.ceil(len(xtrain)/4))
    print(quarter_length_ceil)
    
    global X_test, X_valid, X_train, y_test, y_valid, y_train
        
    X_test = xtrain[-(quarter_length_ceil-1):]
    X_valid = xtrain[half_length:half_length + quarter_length_ceil]
    X_train = xtrain[0:half_length]
    
    y_test = ytrain[-(quarter_length_ceil-1):]
    y_valid = ytrain[half_length:half_length + quarter_length_ceil]
    y_train = ytrain[0:half_length]
     
    traindata = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    validdata = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
    testdata = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    
    traindata = traindata.map(convert_file_image)
    validdata = validdata.map(convert_file_image)
    testdata = testdata.map(convert_file_image)
    
    return traindata, validdata, testdata

In [23]:
training, validation, testing = load_data(input_dir, seg_dir)

649


In [26]:
def context_module_unet(layer, filter_size):
    layer = Conv2D(filter_size, (3, 3), padding = 'same')(layer)
    layer = LeakyReLU(alpha = 0.3)(layer)
    layer = Dropout(0.3)(layer)
    layer = Conv2D(filter_size, (3, 3), padding = 'same')(layer)
    layer = LeakyReLU(alpha = 0.3)(layer)

    return layer

def upsample_module_unet(layer, filter_size):
    layer = UpSampling2D()(layer)
    layer = Conv2D(filter_size, (3, 3), padding = 'same')(layer)
    layer = LeakyReLU(alpha = 0.3)(layer)
    
    return layer
    
def localisation_module_unet(layer, filter_size):
    layer = Conv2D(filter_size, (3, 3), padding = 'same')(layer)
    layer = LeakyReLU(alpha = 0.3)(layer)
    layer = Conv2D(filter_size, (1, 1), padding = 'same')(layer)
    layer = LeakyReLU(alpha = 0.3)(layer)

    return layer

def segmentation_layer(localise_module_a, localise_module_b, conv_a):
    segment_1 = Conv2D(1, (1, 1), padding = 'same')(localise_module_a)
    segment_1 = LeakyReLU(alpha = 0.3)(segment_1)

    upsample_a = UpSampling2D()(segment_1)
    
    segment_2 = Conv2D(1, (1, 1), padding = 'same')(localise_module_b)
    segment_2 = LeakyReLU(alpha = 0.3)(segment_2)
    sum_a = add([upsample_a, segment_2])
    
    upsample_b = UpSampling2D()(sum_a)
    segment_3 = Conv2D(1, (1, 1), padding = 'same')(conv_a)
    segment_3 = LeakyReLU(alpha = 0.3)(segment_3)

    sum_b = add([upsample_b, segment_3])
    
    return sum_b

In [37]:
def unet_model():
    inputs = Input((256, 256, 1))
    
    conv2D_1 = Conv2D(16, (3, 3), padding = 'same')(inputs)
    conv2D_1 = LeakyReLU(alpha = 0.3)(conv2D_1)
    
    cont_1 = context_module_unet(conv2D_1, 16)
    sum_1 = add([conv2D_1, cont_1])
    
    conv2D_2 = Conv2D(32, (3, 3), padding = 'same', strides = 2)(sum_1)
    conv2D_2 = LeakyReLU(alpha = 0.3)(conv2D_2)
    cont_2 = context_module_unet(conv2D_2, 32)
    sum_2 = add([conv2D_2, cont_2])
    
    conv2D_3 = Conv2D(64, (3, 3), padding = 'same', strides = 2)(sum_2)
    conv2D_3 = LeakyReLU(alpha = 0.3)(conv2D_3)
    cont_3 = context_module_unet(conv2D_3, 64)
    sum_3 = add([conv2D_3, cont_3])
    
    conv2D_4 = Conv2D(128, (3, 3), padding = 'same', strides = 2)(sum_3)
    conv2D_4 = LeakyReLU(alpha = 0.3)(conv2D_4)
    cont_4 = context_module_unet(conv2D_4, 128)
    sum_4 = add([conv2D_4, cont_4])
    
    conv2D_5 = Conv2D(256, (3, 3), padding = 'same', strides = 2)(sum_4)
    conv2D_5 = LeakyReLU(alpha = 0.3)(conv2D_5)
    cont_5 = context_module_unet(conv2D_5, 256)
    sum_5 = add([conv2D_5, cont_5])
    
    upsample_1 = upsample_module_unet(sum_5, 128)
    concatenate_1 = concatenate([upsample_1, sum_4])
    
    localise_1 = localisation_module_unet(concatenate_1, 128)
    upsample_2 = upsample_module_unet(localise_1, 64)
    concatenate_2 = concatenate([upsample_2, sum_3])
    
    localise_2 = localisation_module_unet(concatenate_2, 64)
    upsample_3 = upsample_module_unet(localise_2, 32)
    concatenate_3 = concatenate([upsample_3, sum_2])
    
    localise_3 = localisation_module_unet(concatenate_3, 32)
    upsample_4 = upsample_module_unet(localise_3, 16)
    concatenate_4 = concatenate([upsample_4, sum_1])
    
    conv2D_6 = Conv2D(32, (3, 3), padding = 'same')(concatenate_4)
    conv2D_6 = LeakyReLU(alpha = 0.3)(conv2D_6)
    
    segmentation_1 = segmentation_layer(localise_2, localise_3, conv2D_6)
    
    conv2D_final = Conv2D(2, (1, 1), activation = 'softmax', padding = 'same')(segmentation_1)

    model = tf.keras.Model(inputs=inputs, outputs=conv2D_final)

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [38]:
model = unet_model()

results = model.fit(x=training.batch(10), epochs = 30, validation_data = validation.batch(10))

Train for 130 steps, validate for 65 steps
Epoch 1/30
130/130 [==============================] - 79s 609ms/step - loss: 0.3780 - accuracy: 0.8553 - val_loss: 0.4959 - val_accuracy: 0.7812
Epoch 2/30
130/130 [==============================] - 77s 592ms/step - loss: 0.3055 - accuracy: 0.8752 - val_loss: 0.4658 - val_accuracy: 0.7948
Epoch 3/30
130/130 [==============================] - 37s 284ms/step - loss: 0.2653 - accuracy: 0.8999 - val_loss: 0.5600 - val_accuracy: 0.6781
Epoch 4/30
130/130 [==============================] - 77s 593ms/step - loss: 0.2379 - accuracy: 0.9085 - val_loss: 0.4087 - val_accuracy: 0.8056
Epoch 5/30
130/130 [==============================] - 47s 361ms/step - loss: 0.2193 - accuracy: 0.9159 - val_loss: 1.0621 - val_accuracy: 0.5655
Epoch 6/30
130/130 [==============================] - 47s 360ms/step - loss: 0.2074 - accuracy: 0.9202 - val_loss: 0.5856 - val_accuracy: 0.7977
Epoch 7/30
130/130 [==============================] - 77s 592ms/step - loss: 0.1990 - a

KeyboardInterrupt: 

In [ ]:
plt.plot(results.history['accuracy'], label='Training data accuracy')
plt.plot(results.history['val_accuracy'], label = 'Test data accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(x=testing.batch(1))